In [6]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'credentials.json'

In [2]:
! pip install google-cloud-vision

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import csv
import os
from google.cloud import vision
from google.cloud.vision_v1 import types
import io

In [4]:
from google.cloud import vision
from google.cloud.vision_v1 import ImageAnnotatorClient, types, Feature
import io

In [11]:
client = ImageAnnotatorClient()

def detect_features(image_path_or_url):
    """Detect features of an image given a path or URL."""
    if image_path_or_url.startswith('http://') or image_path_or_url.startswith('https://'):
        image = types.Image()
        image.source.image_uri = image_path_or_url
    else:
        with io.open(image_path_or_url, 'rb') as image_file:
            content = image_file.read()
        image = types.Image(content=content)
    
    response = client.annotate_image({
        'image': image,
        'features': [
            Feature(type=Feature.Type.LABEL_DETECTION),
            Feature(type=Feature.Type.TEXT_DETECTION),
            Feature(type=Feature.Type.LOGO_DETECTION),
            Feature(type=Feature.Type.OBJECT_LOCALIZATION),
        ],
    })
    
    labels = [label.description for label in response.label_annotations]
    texts = [text.description for text in response.text_annotations]
    logos = [logo.description for logo in response.logo_annotations]
    objects = [obj.name for obj in response.localized_object_annotations]
    
    return {
        'labels': labels,
        'texts': texts,
        'logos': logos,
        'objects': objects,
    }

import csv

def process_csv(input_csv_path, output_csv_path):
    """Process the input CSV and store the results in an output CSV."""
    with open(input_csv_path, mode='r', encoding='utf-8') as infile, \
         open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.DictReader(infile)
        fieldnames = ['product-name', 'url', 'description']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in reader:
            try:
                product_name = row['product-name']
                url = row['url']
            except KeyError as e:
                print(f"Missing column in input CSV: {e}")
                continue
            
            features = detect_features(url)
            description = f"Labels: {features['labels']}, Texts: {features['texts']}, Logos: {features['logos']}, Objects: {features['objects']}"
            writer.writerow({'product-name': product_name, 'url': url, 'description': description})

process_csv('products.csv', 'results.csv')





In [10]:
# Example usage:
process_csv('products.csv', 'results.csv')

UnicodeEncodeError: 'charmap' codec can't encode character '\u043e' in position 257: character maps to <undefined>

In [16]:

similar_images = find_similar_image('https://image.api.playstation.com/vulcan/ap/rnd/202305/1713/8511ffcc5581ffd7c66faa5f0998344abdb2b04a83e1963e.jpg', 'results.csv', threshold=0.5)
if similar_images:
    for image in similar_images:
        print(f"Matched Product: {image['product-name']}, URL: {image['url']}")
else:
    print("No similar images found.")

Matched Product: assassins-creed-unity, URL: https://rukminim2.flixcart.com/image/850/1000/av-media/games/9/x/z/assassin-s-creed-unity-original-imadx5mqzttcftvj.jpeg?q=20&crop=false
Matched Product: battlefield-2042, URL: https://upload.wikimedia.org/wikipedia/en/e/ec/Battlefield_2042_cover_art.jpg
Matched Product: battlefield-1, URL: https://upload.wikimedia.org/wikipedia/en/f/fc/Battlefield_1_cover_art.jpg
Matched Product: battlefield-4, URL: https://www.gameconnect.net/wp-content/uploads/2013/03/battlefield_4_-_key_art-938x640.jpg
